# Web-map Uglich

**Date**: 17.05.2025

**Team**: Rail' Ibragimov, Maxim Osipenko, Polina Tsikhelashvili


## 1. Импорт библиотек

In [1]:
## For working with dataframes

import pandas as pd
import geopandas as gpd
import folium
import random
import branca.colormap as cm
from folium.plugins import MarkerCluster
from shapely import geometry


## 2.Загружаем наши геоджейсоны и даём им условные названия


In [2]:
okn_poi = gpd.read_file(r'D:\Project_QGIS\labs\3\geojson\okn_centroids.geojson')
okn_poly = gpd.read_file(r'D:\Project_QGIS\labs\3\geojson\okn_polygons.geojson')
okn_indastrial_poi = gpd.read_file(r'D:\Project_QGIS\labs\3\geojson\industrial_heritage.geojson')
main_tr_poi = gpd.read_file(r'D:\Project_QGIS\labs\3\geojson\main_stations.geojson')
bus_stop_poi = gpd.read_file(r'D:\Project_QGIS\labs\3\geojson\bus_stations.geojson')
bus_line = gpd.read_file(r'D:\Project_QGIS\labs\3\geojson\bus_routs.geojson')
food_poi = gpd.read_file(r'D:\Project_QGIS\labs\3\geojson\food_service.geojson')
kult_poi = gpd.read_file(r'D:\Project_QGIS\labs\3\geojson\attractors.geojson')
hotel_poi = gpd.read_file(r'D:\Project_QGIS\labs\3\geojson\hotels.geojson')
agensy_poi = gpd.read_file(r'D:\Project_QGIS\labs\3\geojson\travel_agencies.geojson')
park_poly = gpd.read_file(r'D:\Project_QGIS\labs\3\geojson\public_spaces.geojson')


okn_poi.head()

,fid,NAME,TYPE,geometry
0,3,Ансамбль Алексеевского монастыря,Федерального значения,POINT (4266966.93 7868534.165)
1,7,Ансамбль Алексеевского монастыря,Федерального значения,POINT (4267077.361 7868534.996)
2,8,Ансамбль Алексеевского монастыря,Федерального значения,POINT (4267089.048 7868597.33)
3,14,Ансамбль Алексеевского монастыря,Федерального значения,POINT (4266946.331 7868491.869)
4,16,Ансамбль Алексеевского монастыря,Федерального значения,POINT (4266957.783 7868428.945)


## 3. Определяем границы экстента

In [3]:
# но сначала перепроицируем слои
okn_poi = okn_poi.to_crs(epsg=4326)
okn_poly = okn_poly.to_crs(epsg=4326)
okn_indastrial_poi = okn_indastrial_poi.to_crs(epsg=4326)
main_tr_poi = main_tr_poi.to_crs(epsg=4326)
bus_stop_poi = bus_stop_poi.to_crs(epsg=4326)
bus_line = bus_line.to_crs(epsg=4326)
food_poi = food_poi.to_crs(epsg=4326)
kult_poi = kult_poi.to_crs(epsg=4326)
hotel_poi = hotel_poi.to_crs(epsg=4326)
agensy_poi = agensy_poi.to_crs(epsg=4326)
park_poly = park_poly.to_crs(epsg=4326)

In [4]:
# total_extent = okn_indastrial_poi.total_bounds
# minX, minY, maxX, maxY = total_extent
from shapely.geometry import GeometryCollection
all_geometries = (
    list(okn_poi.geometry) +
    list(okn_poly.geometry) +
    list(okn_indastrial_poi.geometry) +
    list(main_tr_poi.geometry) +
    list(bus_stop_poi.geometry) +
    list(bus_line.geometry) +
    list(food_poi.geometry) +
    list(kult_poi.geometry) +
    list(hotel_poi.geometry) +
    list(agensy_poi.geometry) +
    list(park_poly.geometry)
)

# Находим общие границы
total_extent = GeometryCollection(all_geometries).bounds
minX, minY, maxX, maxY = total_extent

## 4. Создадим карту c отображением в центре total_extent 

In [5]:
center_lat = (minY + maxY) / 2
center_lon = (minX + maxX) / 2
m = folium.Map(location=[57.53, 38.32], zoom_start=13, tiles='cartodb positron', control_scale=True)

## 5. Наполним карту данными которые просто будут отображаться

### 5.1 Данные по ОКН

In [ ]:
m = folium.Map(location=[57.53, 38.32], zoom_start=13, tiles='cartodb positron', control_scale=True)
folium.GeoJson(
    okn_poly,
    name="Объекты культурного наследия",
    tooltip=folium.GeoJsonTooltip(fields=['NAME'], aliases=['Название:']),
    popup=folium.GeoJsonPopup(fields=['NAME', 'TYPE'], aliases=['Название:', 'Статус:'])
).add_to(m)


### 5.2 Станции междугороднего транспорта

In [7]:
main_tr_layer = folium.FeatureGroup(name='Станции междугороднего транспорта')

for idx, row in main_tr_poi.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        color='red',
        fill=True,
        fill_opacity=0.7,
        tooltip=f"<b>Название:</b> {row.get('NAME', 'Не указано')}",
        popup=folium.Popup(f"<b>Название:</b> {row.get('NAME', 'Не указано')}", max_width=250)
    ).add_to(main_tr_layer)

main_tr_layer.add_to(m)


### 5.3 Автобусные остановки

In [8]:
bus_stop_layer = folium.FeatureGroup(name='Остановки автобусов')

for idx, row in bus_stop_poi.iterrows():
    popup_html = (
        f"<b>Название:</b> {row.get('title', 'Не указано')}<br>"
        f"<b>Маршруты:</b> {row.get('routes_labes', 'Не указано')}<br>"
        f"<b>Я.Карты:</b> {row.get('direct_link', 'Не указано')}"
    )
    
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        color='green',
        fill=True,
        fill_opacity=0.7,
        tooltip=f"{row.get('title', 'Не указано')}",
        popup=folium.Popup(popup_html, max_width=250)
    ).add_to(bus_stop_layer)

bus_stop_layer.add_to(m)



### 5.4 Автобусные линии

In [9]:
from folium.plugins import GroupedLayerControl

# Создаем карту, центрируем её (например, по координатам центра bus_line)
m = folium.Map(location=[bus_line.geometry.centroid.y.mean(), bus_line.geometry.centroid.x.mean()], zoom_start=12)

# Словарь для групп слоёв (ключ — имя группы, значение — список слоёв)
groups_dict = {}

# Добавим маршруты из bus_line, каждый в отдельную группу с именем маршрута
for idx, row in bus_line.iterrows():
    route_name = f"Маршрут {row['name']}"
    line_layer = folium.FeatureGroup(name=route_name, show=False)  # show=False чтобы не показывать все сразу
    
    folium.GeoJson(
        row.geometry,
        style_function=lambda feature: {
            'color': 'blue',
            'weight': 4
        },
        tooltip=route_name,
        popup=folium.Popup(route_name, max_width=250)
    ).add_to(line_layer)
    
    line_layer.add_to(m)
    
    # Добавляем в словарь групп (ключ — имя группы, значение — список из соответствующих слоёв)
    groups_dict.setdefault('Маршруты автобусов', []).append(line_layer)

# Добавляем контрол групп слоёв
GroupedLayerControl(
    groups=groups_dict,
    collapsed=True,   # меню свернуто по умолчанию
    position='topleft' # левый верхний угол
).add_to(m)

# Отобразить карту

C:\Users\osipe\AppData\Local\Temp\ipykernel_12228\4048238021.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[bus_line.geometry.centroid.y.mean(), bus_line.geometry.centroid.x.mean()], zoom_start=12)


### 5.5 Кластарезация по общепиту

In [10]:
colormap = cm.LinearColormap(['red', 'yellow', 'green'], vmin=1, vmax=5)
colormap.caption = 'Рейтинг'

food_feature_group = folium.FeatureGroup(name='Кафе, рестораны')

food_cluster = MarkerCluster()

for idx, row in food_poi.iterrows():
    lat, lon = row.geometry.y, row.geometry.x
    rating = row.get('rating_value', 1)
    color = colormap(rating)

    popup_html = f"""
    <b>Название:</b> {row.get('title', 'Не указано')}<br>
    <b>Тип:</b> {row.get('rubric', 'Не указано')}<br>
    <b>Рабочее время:</b> {row.get('working_time', 'Не указано')}<br>
    <b>Рейтинг:</b> {rating}<br>
    <b><a href="{row.get('direct_link', '#')}" target="_blank">Я.Карты</a></b>
    """

    folium.CircleMarker(
        location=[lat, lon],
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"{row.get('title', 'Не указано')} — Рейтинг: {rating}"
    ).add_to(food_cluster)

food_cluster.add_to(food_feature_group)
food_feature_group.add_to(m)

colormap.add_to(m)


### 5.6 Тепловая карта по местам досуга

In [11]:
from folium.plugins import HeatMap

# Создаём слой для тепловой карты
heatmap_layer = folium.FeatureGroup(name='Места досуга')

# Формируем список координат в формате [lat, lon]
heat_data = [[point.y, point.x] for point in kult_poi.geometry if point is not None]

# Добавляем тепловую карту в слой
HeatMap(heat_data, radius=15, blur=10, max_zoom=13).add_to(heatmap_layer)

# Добавляем слой на карту
heatmap_layer.add_to(m)

### 5.7 ТурАгенства

In [12]:
agency_layer = folium.FeatureGroup(name='Туристические агентства')

for idx, row in agensy_poi.iterrows():
    name = row.get('title', 'Без названия')
    working_time = row.get('working_time', 'Режим работы не указан')
    rating = row.get('rating_value', 'Нет рейтинга')
    direct_link = row.get('direct_link', '#')

    tooltip_text = f"{name} — {working_time}"

    popup_html = f"""
    <b>Название:</b> {name}<br>
    <b>Режим работы:</b> {working_time}<br>
    <b>Рейтинг:</b> {rating}<br>
    <b><a href="{direct_link}" target="_blank">Я.Карты</a></b>
    """

    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=6,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.7,
        tooltip=tooltip_text,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(agency_layer)

agency_layer.add_to(m)

### 5.8 Гостиницы (градюирование точек в записимости от рейтинга)

In [13]:
hotel_layer = folium.FeatureGroup(name='Гостиницы')

for idx, row in hotel_poi.iterrows():
    name = row.get('name', 'Без названия')
    rating = row.get('rating_value', None)
    site_link = row.get('site', '#')
    yandex_link = row.get('direct_link', '#')
    
    if rating is None or not isinstance(rating, (float, int)):
        rating = 1.0  # дефолтный рейтинг
    
    color = colormap(rating)
    tooltip_text = f"{name} — Рейтинг: {rating:.2f}"
    
    popup_html = f"""
    <b>Название:</b> {name}<br>
    <b>Рейтинг:</b> {rating:.2f}<br>
    <b><a href="{site_link}" target="_blank">Сайт гостиницы</a></b><br>
    <b><a href="{yandex_link}" target="_blank">Я.Карты</a></b>
    """
    
    lat, lon = row.geometry.y, row.geometry.x
    radius = 15
    
    # Добавляем кружок
    folium.CircleMarker(
        location=[lat, lon],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        tooltip=tooltip_text,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(hotel_layer)
    
    # Добавляем цифру с рейтингом по центру кружка
    rating_rounded = round(rating)
    folium.map.Marker(
        [lat, lon],
        icon=folium.DivIcon(html=f"""
            <div style="
                display:flex;
                justify-content:center;
                align-items:center;
                width:{radius*2}px;
                height:{radius*2}px;
                border-radius:{radius}px;
                background-color:{color};
                color: white;
                font-weight: bold;
                font-size: 14px;
                border: 1.5px solid white;
                box-shadow: 0 0 2px #000;
                ">
                {rating_rounded}
            </div>
        """)
    ).add_to(hotel_layer)

hotel_layer.add_to(m)

### 5.9 Парки и прогулочные зоны

In [14]:
park_layer = folium.FeatureGroup(name='Парки и прогулочные зоны')

folium.GeoJson(
    park_poly,
    style_function=lambda feature: {
        'fillColor': '#9ACD32',      # салатовый цвет заливки
        'color': '#9ACD32',          # салатовый цвет границ
        'weight': 3,                 # толщина границ
        'fillOpacity': 0.4           # прозрачность заливки
    },
    tooltip=folium.GeoJsonTooltip(fields=['NAME'], aliases=['Название:']),
).add_to(park_layer)

park_layer.add_to(m)

### 5.10 Создаём мини-карту

In [16]:
from folium.plugins import MiniMap

minimap = MiniMap(toggle_display=True, position='bottomright')
m.add_child(minimap)

## 6. Выводим легенду

In [17]:
folium.LayerControl().add_to(m)

# Туристическая карта Углича

In [18]:
m